# COURSE: Master Python for scientific programming by solving projects
## PROJECT: Spectral analysis
#### TEACHER: Mike X Cohen, sincxpress.com
##### COURSE URL: udemy.com/course/maspy_x/?couponCode=202201

In [ ]:
# import all necessary modules
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch

# Simulate an AR process

In [ ]:
# temporal weighting vector
w = np.array([-.6,.9]) # -2,-1
k = len(w)

N = 200
x = np.zeros(N)

for i in range(k,N):
  x[i] = sum( w*x[i-k:i] ) + np.random.randn()

# add sine wave
x += np.sin(np.linspace(0,10*np.pi,N))

# plot the signal
plt.plot(x)
plt.xlabel('X axis')
plt.ylabel('Value')
plt.title('AR(%g) process'%k)
plt.show()

# Code the Fourier transform

In [ ]:
# time vector
t = np.arange(0,N)/N

# initialize Fourier coefficients
fc = np.zeros(N,dtype=complex)

for f in range(N):
  # create complex sine wave
  csw = np.exp(-1j*2*np.pi*f*t)

  # dot product with signal
  fc[f] = np.dot(csw,x)

In [ ]:
# check against the fft
fc2 = np.fft.fft(x)

In [ ]:
# normalized frequencies vector
hz = np.linspace(0,1,int(N/2+1))

# and plot
plt.plot(hz,np.abs(fc[:len(hz)]),label='Manual FT')
plt.plot(hz,np.abs(fc2[:len(hz)]),'ro',label='FFT')
plt.xlabel('Frequency (fraction of Nyquist)')
plt.ylabel('Amplitude (a.u.)')
plt.legend()
plt.show()

# Zero-padding the FFT

In [ ]:
# new signal with N=21

w = [.9,.6]
k = len(w)

N = 21
x = np.zeros(N)

for i in range(k,N):
  x[i] = sum( w*x[i-k:i] ) + np.random.randn()

In [ ]:
# for-loop to run FFT and plot power spectrum

for i in range(3):
  nfft = N + 10**i * int(i!=0)
  print(nfft)
  xX = np.fft.fft(x,n=nfft)
  
  # normalized frequencies vector
  hz = np.linspace(0,1,int(nfft/2)+1)
  
  # and plot
  plt.plot(hz,np.abs(xX[:len(hz)])**2,'.-',label='%s-point FT'%nfft)
  
plt.xlabel('Frequency (fraction of Nyquist)')
plt.ylabel('Power (a.u.)')
plt.legend()
plt.show()

# Welch's method

In [ ]:
# signal with phase reversal

N = 2000
t = np.linspace(0,4*np.pi,N)
signal1 = np.concatenate((np.sin(t), np.sin(t)))
signal2 = np.concatenate((np.sin(t),-np.sin(t)))

plt.plot(signal1)
plt.plot(signal2)
plt.show()

In [ ]:
# "static" FFT
staticX1 = np.fft.fft(signal1)
staticX2 = np.fft.fft(signal2)
staticHz = np.linspace(0,1,int(len(signal1)/2+1))

# "dynamic" FFT via welch's function
dynamicHz,dynamicX1 = welch(signal1,nfft=N)
dynamicX2 = welch(signal2,nfft=N)[1]

# And plot
fig,ax = plt.subplots(1,2,figsize=(13,4))

ax[0].plot(staticHz,np.abs(staticX1[:len(staticHz)]),'-o',label='Signal 1')
ax[0].plot(staticHz,np.abs(staticX2[:len(staticHz)]),'-o',label='Signal 2')
ax[0].legend()
ax[0].set_xlim([0,.01])
ax[0].set_title('Static spectrum via FFT')

ax[1].plot(dynamicHz,np.abs(dynamicX1[:len(dynamicHz)]),'-o',label='Signal 1')
ax[1].plot(dynamicHz,np.abs(dynamicX2[:len(dynamicHz)]),'-o',label='Signal 2')
ax[1].legend()
ax[1].set_xlim([0,.01])
ax[1].set_title('Dynamic spectrum via Welch')

plt.show()

# Bonus: Spectrogram

In [ ]:
from scipy.signal import spectrogram

f, t, Sxx = spectrogram(signal2,noverlap=10,nperseg=1000)
plt.pcolormesh(t,f,Sxx,shading='gouraud')
plt.ylabel('Frequency (a.u.)')
plt.xlabel('Time (a.u.)')
plt.ylim([0,.005])
plt.show()
